In [ ]:
import os
from llama_index.core import (
    load_index_from_storage,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

EMBEDDING_DIMENSIONS = {
    "sentence-transformers/all-MiniLM-L6-v2": 384,
    "sentence-transformers/all-mpnet-base-v2": 768,
    "text-embedding-3-small": 1536,
}

In [35]:
input_dir = "/Users/goeksu/dev/interactive_rag/data/landing"
persist_root = "/Users/goeksu/dev/interactive_rag/data/index"
chunk_size = 256
model_name = "text-embedding-3-small"
persist_dir = os.path.join(persist_root, model_name)
os.makedirs(persist_dir, exist_ok=True)

# File ingestion
# Simple directory reader seems to be sufficient for this task.
reader = SimpleDirectoryReader(input_dir=input_dir)
documents = reader.load_data()
# Split docs to nodes and create vector index
# Just setting a sliding window chunk here, nothing fancy.
node_parser = SimpleNodeParser.from_defaults(
    chunk_size=chunk_size, chunk_overlap=int(chunk_size * 0.2)
)
nodes = node_parser.get_nodes_from_documents(documents=documents)

In [19]:
# Initialize embedding model
if model_name.startswith("text-embedding-3"):
    embedding_model = OpenAIEmbedding(model=model_name)
else:
    embedding_model = HuggingFaceEmbedding(model_name=model_name)

In [ ]:
# Initialize a faiss vector store
index = VectorStoreIndex(nodes=nodes, embed_model=embedding_model)

# d = EMBEDDING_DIMENSIONS[model_name]
# faiss_index = faiss.IndexFlatL2(d)
# vector_store = FaissVectorStore(faiss_index=faiss_index)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex(
#    nodes=nodes, storage_context=storage_context, embed_model=embedding_model
# )

# Save index to disk
index.storage_context.persist(persist_dir=persist_dir)

In [24]:
# interact with index
chat_engine = index.as_chat_engine(llm=OpenAI(model="gpt-4o-mini"), similarity_top_k=3)

In [25]:
response = chat_engine.chat("What sort of information about ChemNovus can you provide?")
print(response)

ChemNovus Incorporated provides various employee benefits, including:

- Competitive salaries
- Comprehensive health insurance
- Tuition assistance for job-related education
- Discounts on various products and services

The company also promotes charitable giving and community involvement through volunteer programs and matching gift initiatives. It's important to note that specific benefits and eligibility may vary based on location and local agreements. For detailed inquiries, employees are encouraged to contact their local Human Resources department or access the Employee Benefits Portal.


In [26]:
response = chat_engine.chat("Can you give me more information about health benefits?")
print(response)

ChemNovus Incorporated offers a variety of health and wellness benefits to its employees, which include:

### United States
- **Medical Plan Options**: Employees can choose from three medical plans through a partnership with TheInsuranceCompany Ltd. These plans cover:
  - Preventive care
  - Doctor visits
  - Hospital stays
  - Prescription drugs
- **Plan Variations**: The plans differ in terms of deductibles, copays, and coinsurance rates. Options include:
  - A traditional PPO plan with a broad network and moderate costs.
  - A high-deductible health plan that allows contributions to a health savings account (HSA).

### European Region
- **Statutory Health Insurance**: Participation in the statutory health insurance system is typically required, providing comprehensive medical coverage. ChemNovus contributes to the premiums, while employees cover the remaining costs through payroll deductions.
- **Supplementary Health Insurance**: Additional options are available to enhance coverage,

In [36]:
# Load index and interact
# vector_store = FaissVectorStore.from_persist_dir(persist_dir)
# storage_context = StorageContext.from_defaults(
#    vector_store=vector_store, persist_dir=persist_dir
# )
# index_load = load_index_from_storage(storage_context=storage_context)

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
# load index
index_load = load_index_from_storage(storage_context, embed_model=embedding_model)

# interact with index
chat_engine_2 = index_load.as_chat_engine(
    llm=OpenAI(model="gpt-4o-mini"),
    #     chat_mode="condense_plus_context",
    similarity_top_k=3,
)

In [37]:
response = chat_engine_2.chat(
    "What sort of information about ChemNovus can you provide?"
)
print(response)

ChemNovus Incorporated provides a variety of employee benefits, which include:

- Competitive salaries
- Health insurance
- Tuition assistance
- Employee discounts
- Support for charitable giving and volunteer programs

Employees may also be eligible for discounts on various products and services, and the company promotes participation in community activities. It's important to note that specific benefits and eligibility criteria can vary by location and local agreements. For more detailed information, employees are encouraged to reach out to their local Human Resources department or check the Employee Benefits Portal.
